In [1]:
from datetime import date
import time
from assets.classTypes import Category
from assets.scrape import UseBeautifulSoup as useScrape
from assets.adScrape import advertisementScrape as useAdScrape
from assets.spliter import createLinkList, splitUrl

initialUrl = 'https://www.zangia.mn/'
categorySet = set()
adUrlDict = {}
adsSet = set()
conn = None
cursor = None

In [7]:

# scrape initial links
soup = useScrape(initialUrl)
navigatorList = soup.find_all('div', class_='filter')
for navigator in navigatorList:
    if navigator.find('h3').text.strip() != 'Салбар, мэргэжил':
        continue
    # ALL CATEGORY LINKS
    categoryList = navigator.find_all('div')

for categoryItem in categoryList:
    categories = categoryItem.find('a')
    url = initialUrl + categories['href']
    tempCategory = Category(splitUrl(url, 'b.'), url, categories.text)
    print('CATEGORY LINK SCRAPED! ', url, tempCategory.id)
    soup = useScrape(url)
    subCategory = soup.find('div', class_='pros')
    # ALL SUBCATEGORY LINKS
    subCategoryList = subCategory.find_all('a')
    for subCategoryItem in subCategoryList:
        subCategoryUrl = initialUrl + subCategoryItem['href']
        tempSubCategory = Category(splitUrl(subCategoryUrl, 'r.'),
                                   subCategoryUrl, subCategoryItem.text, tempCategory)
        print(tempSubCategory.id, tempSubCategory.parentCategory.id)
        categorySet.add(tempSubCategory)
    categorySet.add(tempCategory)

CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.17 b.17
r.1204 b.17
r.479 b.17
r.481 b.17
r.1201 b.17
r.927 b.17
r.1202 b.17
r.779 b.17
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.33 b.33
r.843 b.33
r.1157 b.33
r.850 b.33
r.1154 b.33
r.1156 b.33
r.1065 b.33
r.1160 b.33
r.852 b.33
r.855 b.33
r.848 b.33
r.1159 b.33
r.1281 b.33
r.857 b.33
r.1331 b.33
r.1315 b.33
r.844 b.33
r.859 b.33
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.19 b.19
r.1003 b.19
r.486 b.19
r.1300 b.19
r.489 b.19
r.493 b.19
r.495 b.19
r.488 b.19
r.791 b.19
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.20 b.20
r.925 b.20
r.514 b.20
r.1230 b.20
r.507 b.20
r.826 b.20
r.744 b.20
r.1242 b.20
r.1232 b.20
r.838 b.20
r.1311 b.20
r.1235 b.20
r.827 b.20
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.22 b.22
r.1383 b.22
r.1005 b.22
r.538 b.22
r.531 b.22
r.537 b.22
r.1030 b.22
r.536 b.22
r.1243 b.22
r.534 b.22
r.532 b.22
r.794 b.22
r.540 b.22
r.1304 b.22
r.1238 b.22
r.539 b.22
r

In [8]:
len(categorySet)

349

In [61]:
try:
  conn = psycopg2.connect(database="chatbot",
                        user='chatbot', password='1234!@#$',
                        host='3.228.127.116', port='5432'
                        )
  conn.autocommit = True
  cursor = conn.cursor()
  cursor.execute(create_category_script)
  cursor.execute(create_advertisement_script)
  
  def upsertCategory (cat: Category):
    if (cat.parentCategory != None):
      upsertCategory(cat.parentCategory)
      
    r = cursor.execute("select * from categry where external_id = ?", cat.externalId)
    if (r != None):
      cursor.execute('update category set name = ? where id = ?', r.name, r.id)
    else:
      id = cursor.execute('insert')
      cat.id = id
  for categoryItem in categorySet:
    # insert_value = (categoryItem.url, categoryItem.name)
    # cursor.execute(insert_category_script, insert_value)
    upsertCategory(categoryItem)

  conn.commit()
  

except Exception as error:
  print(error)

finally:
  if cursor is not None:
    cursor.close()
  if conn is not None:
    conn.close()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
